In [16]:
import pandas as pd
import polars as pl
import time
import os

In [4]:
data_oct = '../data/2019-Oct.csv'
data_nov = '../data/2019-Nov.csv'

timing_path = '../stats/timings.csv'

In [189]:
if os.path.exists(timing_path):
    timing = pd.read_csv(timing_path, index_col=0)
else:
    timing = pd.DataFrame(columns= ['Timing_pandas', 'Timing_polars'])

In [190]:
timing

,Timing_pandas,Timing_polars
1,83.40,NaN
2,8.81,NaN
3,6.31,NaN
4,6.10,NaN
5,6.76,NaN
6,0.83,NaN
7,5.58,NaN
8,4.33,NaN
9,10.72,NaN
10,28.13,NaN


1. Lecture des deux jeux de données et concatenation dans un seul dataset

In [191]:
ex = 1
start_time = time.time()

raw_data_oct = pl.read_csv(data_oct)
raw_data_nov = pl.read_csv(data_nov)

data = pl.concat([raw_data_oct, raw_data_nov])

end_time = time.time()
execution_time = round(end_time - start_time, 2)
timing.loc[ex, 'Timing_polars'] = execution_time

print(f"Temps d'exécution : {execution_time} secondes")

Temps d'exécution : 32.42 secondes


2.	Affichage d'informations basiques du jeu de données : quantité de lignes, colonnes, types des données, etc.

In [192]:
ex = 2
start_time = time.time()
print(f"Quantité des lignes : {data.height}")

print(data.head())

print(data.dtypes)

print(data.describe())

end_time = time.time()
execution_time = round(end_time - start_time, 2)
timing.loc[ex, 'Timing_polars'] = execution_time

print(f"Temps d'exécution : {execution_time} secondes")

Quantité des lignes : 109950743
shape: (5, 9)
┌────────────┬────────────┬───────────┬───────────┬───┬──────────┬─────────┬───────────┬───────────┐
│ event_time ┆ event_type ┆ product_i ┆ category_ ┆ … ┆ brand    ┆ price   ┆ user_id   ┆ user_sess │
│ ---        ┆ ---        ┆ d         ┆ id        ┆   ┆ ---      ┆ ---     ┆ ---       ┆ ion       │
│ str        ┆ str        ┆ ---       ┆ ---       ┆   ┆ str      ┆ f64     ┆ i64       ┆ ---       │
│            ┆            ┆ i64       ┆ i64       ┆   ┆          ┆         ┆           ┆ str       │
╞════════════╪════════════╪═══════════╪═══════════╪═══╪══════════╪═════════╪═══════════╪═══════════╡
│ 2019-10-01 ┆ view       ┆ 44600062  ┆ 210380745 ┆ … ┆ shiseido ┆ 35.79   ┆ 541312140 ┆ 72d76fde- │
│ 00:00:00   ┆            ┆           ┆ 959538772 ┆   ┆          ┆         ┆           ┆ 8bb3-4e00 │
│ UTC        ┆            ┆           ┆ 4         ┆   ┆          ┆         ┆           ┆ -8c23-a03 │
│            ┆            ┆           ┆      

3.	Affichage de la quantité de produits, marques et utilisateurs dans le dataset, sans répétitions

In [193]:
ex = 3
start_time = time.time()

print(f"Quantité de produits uniques dans le dataset : {data.select(['product_id']).unique().height}")
print(f"Quantité de marques uniques dans le dataset : {data.select(['brand']).unique().height}")
print(f"Quantité d'utilisateurs uniques dans le dataset : {data.select(['user_id']).unique().height}")

end_time = time.time()
execution_time = round(end_time - start_time, 2)
timing.loc[ex, 'Timing_polars'] = execution_time

print(f"Temps d'exécution : {execution_time} secondes")

Quantité de produits uniques dans le dataset : 206876
Quantité de marques uniques dans le dataset : 4304
Quantité d'utilisateurs uniques dans le dataset : 5316649
Temps d'exécution : 5.76 secondes


4. Affichage du top 5 de marques et catégories avec le prix moyen le plus élevé

In [194]:
ex = 4
start_time = time.time()

result_brand = data.group_by('brand').agg(avg_price = pl.col('price').mean().round(2))
result_brand = result_brand.sort(by='avg_price', descending=True)
print(result_brand.head())

result_category = data.group_by('category_id').agg(avg_price = pl.col('price').mean().round(2))
result_category = result_category.sort(by='avg_price', descending=True)
print(result_category.head())

end_time = time.time()
execution_time = round(end_time - start_time, 2)
timing.loc[ex, 'Timing_polars'] = execution_time

print(f"Temps d'exécution : {execution_time} secondes")

shape: (5, 2)
┌─────────────┬───────────┐
│ brand       ┆ avg_price │
│ ---         ┆ ---       │
│ str         ┆ f64       │
╞═════════════╪═══════════╡
│ pinarello   ┆ 2533.24   │
│ nordictrack ┆ 2522.59   │
│ kessler     ┆ 2372.0    │
│ yjfitness   ┆ 2344.98   │
│ climadiff   ┆ 2261.95   │
└─────────────┴───────────┘
shape: (5, 2)
┌─────────────────────┬───────────┐
│ category_id         ┆ avg_price │
│ ---                 ┆ ---       │
│ i64                 ┆ f64       │
╞═════════════════════╪═══════════╡
│ 2104564977263182827 ┆ 1673.14   │
│ 2059484601444729123 ┆ 1666.93   │
│ 2053013564523807647 ┆ 1527.04   │
│ 2144356630513320518 ┆ 1489.79   │
│ 2179887855134835524 ┆ 1381.51   │
└─────────────────────┴───────────┘
Temps d'exécution : 2.93 secondes


5.	Calcul du min, du max et de la moyenne de prix par marque et par catégorie

In [195]:
ex = 5
start_time = time.time()

result_brand_price = data.group_by('brand').agg(
    min_price = pl.col('price').mean().round(2),
    max_price = pl.col('price').mean().round(2),
    avg_price = pl.col('price').mean().round(2))
print(result_brand_price.head())

result_category_price = data.group_by('category_id').agg(
    min_price = pl.col('price').mean().round(2),
    max_price = pl.col('price').mean().round(2),
    avg_price = pl.col('price').mean().round(2))
print(result_category_price.head())

end_time = time.time()
execution_time = round(end_time - start_time, 2)
timing.loc[ex, 'Timing_polars'] = execution_time

print(f"Temps d'exécution : {execution_time} secondes")

shape: (5, 4)
┌───────────┬───────────┬───────────┬───────────┐
│ brand     ┆ min_price ┆ max_price ┆ avg_price │
│ ---       ┆ ---       ┆ ---       ┆ ---       │
│ str       ┆ f64       ┆ f64       ┆ f64       │
╞═══════════╪═══════════╪═══════════╪═══════════╡
│ glanzen   ┆ 5.96      ┆ 5.96      ┆ 5.96      │
│ millenium ┆ 72.07     ┆ 72.07     ┆ 72.07     │
│ buggy     ┆ 241.86    ┆ 241.86    ┆ 241.86    │
│ veho      ┆ 25.41     ┆ 25.41     ┆ 25.41     │
│ tissot    ┆ 648.43    ┆ 648.43    ┆ 648.43    │
└───────────┴───────────┴───────────┴───────────┘
shape: (5, 4)
┌─────────────────────┬───────────┬───────────┬───────────┐
│ category_id         ┆ min_price ┆ max_price ┆ avg_price │
│ ---                 ┆ ---       ┆ ---       ┆ ---       │
│ i64                 ┆ f64       ┆ f64       ┆ f64       │
╞═════════════════════╪═══════════╪═══════════╪═══════════╡
│ 2053013559071212141 ┆ 1.01      ┆ 1.01      ┆ 1.01      │
│ 2134905044833666047 ┆ 278.78    ┆ 278.78    ┆ 278.78    │
│ 

6.	Affichage des 5 prix les plus élevés

In [196]:
ex = 6
start_time = time.time()

result_price = data.select(['price']).unique().sort(by='price', descending=True)
print(result_price.head())

end_time = time.time()
execution_time = round(end_time - start_time, 2)
timing.loc[ex, 'Timing_polars'] = execution_time

print(f"Temps d'exécution : {execution_time} secondes")

shape: (5, 1)
┌─────────┐
│ price   │
│ ---     │
│ f64     │
╞═════════╡
│ 2574.07 │
│ 2574.04 │
│ 2573.99 │
│ 2573.81 │
│ 2573.79 │
└─────────┘
Temps d'exécution : 0.4 secondes


7.	Affichage de la quantité d’articles par marque et par catégorie, en utilisant une fonction d’agrégation

In [197]:
ex = 7
start_time = time.time()

result_brand_qty = data.group_by('brand').agg(qty_articles = pl.col('price').count())
print(result_brand_qty.head())

result_category_qty = data.group_by('category_id').agg(qty_articles = pl.col('price').count())
print(result_category_qty.head())

end_time = time.time()
execution_time = round(end_time - start_time, 2)
timing.loc[ex, 'Timing_polars'] = execution_time

print(f"Temps d'exécution : {execution_time} secondes")

shape: (5, 2)
┌─────────────┬──────────────┐
│ brand       ┆ qty_articles │
│ ---         ┆ ---          │
│ str         ┆ u32          │
╞═════════════╪══════════════╡
│ noktamakro  ┆ 3670         │
│ wowwee      ┆ 1006         │
│ tdmelectric ┆ 52           │
│ gamerstorm  ┆ 54           │
│ bulgari     ┆ 22258        │
└─────────────┴──────────────┘
shape: (5, 2)
┌─────────────────────┬──────────────┐
│ category_id         ┆ qty_articles │
│ ---                 ┆ ---          │
│ i64                 ┆ u32          │
╞═════════════════════╪══════════════╡
│ 2053013556487520725 ┆ 162057       │
│ 2053013559440310913 ┆ 14079        │
│ 2053013552989470973 ┆ 55932        │
│ 2053013558945383017 ┆ 132729       │
│ 2053013559071212141 ┆ 2480         │
└─────────────────────┴──────────────┘
Temps d'exécution : 2.83 secondes


8. Affichage de la quantité d’événements par type, en utilisant une fonction d’agrégation

In [198]:
ex = 8
start_time = time.time()

result_event_type = data.group_by('event_type').agg(qty_events = pl.col('event_time').count())
print(result_event_type.head())

end_time = time.time()
execution_time = round(end_time - start_time, 2)
timing.loc[ex, 'Timing_polars'] = execution_time

print(f"Temps d'exécution : {execution_time} secondes")

shape: (3, 2)
┌────────────┬────────────┐
│ event_type ┆ qty_events │
│ ---        ┆ ---        │
│ str        ┆ u32        │
╞════════════╪════════════╡
│ cart       ┆ 3955446    │
│ view       ┆ 104335509  │
│ purchase   ┆ 1659788    │
└────────────┴────────────┘
Temps d'exécution : 2.25 secondes


9.	Création et affichage d'un tableau contenant la quantité d’événements de chaque type par marque

In [199]:
ex = 9
start_time = time.time()

result_event_type_brand = data.group_by(['event_type', 'brand']).agg(qty_events = pl.col('event_time').count())
print(result_event_type_brand.head())

end_time = time.time()
execution_time = round(end_time - start_time, 2)
timing.loc[ex, 'Timing_polars'] = execution_time

print(f"Temps d'exécution : {execution_time} secondes")

shape: (5, 3)
┌────────────┬─────────────┬────────────┐
│ event_type ┆ brand       ┆ qty_events │
│ ---        ┆ ---         ┆ ---        │
│ str        ┆ str         ┆ u32        │
╞════════════╪═════════════╪════════════╡
│ view       ┆ baden       ┆ 395551     │
│ purchase   ┆ xiaomi      ┆ 124908     │
│ view       ┆ tcl         ┆ 119998     │
│ view       ┆ plantronics ┆ 30935      │
│ view       ┆ m-audio     ┆ 13188      │
└────────────┴─────────────┴────────────┘
Temps d'exécution : 1.17 secondes


10. Création et affichage d'un tableau contenant  la quantité d’événements de chaque type par marque et utilisateur

In [200]:
ex = 10
start_time = time.time()

result_event_type_brand_user = data.group_by(['event_type', 'brand', 'user_id']).agg(qty_events = pl.col('event_time').count())
print(result_event_type_brand_user.head())

end_time = time.time()
execution_time = round(end_time - start_time, 2)
timing.loc[ex, 'Timing_polars'] = execution_time

print(f"Temps d'exécution : {execution_time} secondes")

shape: (5, 4)
┌────────────┬─────────┬───────────┬────────────┐
│ event_type ┆ brand   ┆ user_id   ┆ qty_events │
│ ---        ┆ ---     ┆ ---       ┆ ---        │
│ str        ┆ str     ┆ i64       ┆ u32        │
╞════════════╪═════════╪═══════════╪════════════╡
│ view       ┆ maxwell ┆ 539194858 ┆ 1          │
│ view       ┆ oasis   ┆ 512796782 ┆ 1          │
│ view       ┆ apple   ┆ 517062545 ┆ 3          │
│ view       ┆ null    ┆ 514933060 ┆ 6          │
│ view       ┆ xiaomi  ┆ 521149112 ┆ 4          │
└────────────┴─────────┴───────────┴────────────┘
Temps d'exécution : 4.57 secondes


11. Affichage du pourcentage de données vides ou nulles (valeurs manquantes) par colonne

In [201]:
ex = 11
start_time = time.time()

print(data.null_count()*100/data.height)

end_time = time.time()
execution_time = round(end_time - start_time, 2)
timing.loc[ex, 'Timing_polars'] = execution_time

print(f"Temps d'exécution : {execution_time} secondes")

shape: (1, 9)
┌────────────┬────────────┬────────────┬────────────┬───┬───────────┬───────┬─────────┬────────────┐
│ event_time ┆ event_type ┆ product_id ┆ category_i ┆ … ┆ brand     ┆ price ┆ user_id ┆ user_sessi │
│ ---        ┆ ---        ┆ ---        ┆ d          ┆   ┆ ---       ┆ ---   ┆ ---     ┆ on         │
│ f64        ┆ f64        ┆ f64        ┆ ---        ┆   ┆ f64       ┆ f64   ┆ f64     ┆ ---        │
│            ┆            ┆            ┆ f64        ┆   ┆           ┆       ┆         ┆ f64        │
╞════════════╪════════════╪════════════╪════════════╪═══╪═══════════╪═══════╪═════════╪════════════╡
│ 0.0        ┆ 0.0        ┆ 0.0        ┆ 0.0        ┆ … ┆ 13.943738 ┆ 0.0   ┆ 0.0     ┆ 0.000011   │
└────────────┴────────────┴────────────┴────────────┴───┴───────────┴───────┴─────────┴────────────┘
Temps d'exécution : 0.02 secondes


12. Suppression de lignes vides sur l’ensemble de colonnes où le total de valeurs manquants ne dépasse pas un 15%

In [202]:
ex = 12
start_time = time.time()

total = data.height
print(f"Lignes avant suppression : {total}")

col_to_del = []
for col in data.columns:
    is_na_per = (data.select(pl.col(col)).null_count()*100/total).item()
    if is_na_per != 0 and is_na_per <= 15:
        col_to_del.append(col)
    
print(f"Colonnes à filtrer : {col_to_del}")
filtered_data = data.drop_nulls(subset=col_to_del)

total = filtered_data.height
print(f"Lignes après suppression : {total}")

end_time = time.time()
execution_time = round(end_time - start_time, 2)
timing.loc[ex, 'Timing_polars'] = execution_time

print(f"Temps d'exécution : {execution_time} secondes")

Lignes avant suppression : 109950743
Colonnes à filtrer : ['brand', 'user_session']
Lignes après suppression : 94619490
Temps d'exécution : 13.25 secondes


In [203]:
timing

,Timing_pandas,Timing_polars
1,83.40,32.42
2,8.81,8.10
3,6.31,5.76
4,6.10,2.93
5,6.76,3.10
6,0.83,0.40
7,5.58,2.83
8,4.33,2.25
9,10.72,1.17
10,28.13,4.57


In [204]:
timing.to_csv(timing_path)